In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# --- PARAMETRI ---
CSV_FILE = "dataset_unificato.csv"
SEQUENCE_LENGTH = 100  # Numero di frame per sequenza
FEATURES_PER_FRAME = 132  # 33 landmarks * 4 (x, y, z, visibility)

# --- CARICAMENTO CSV ---
df = pd.read_csv(CSV_FILE)

# --- RAGGRUPPA IN SEQUENZE ---
sequences = []
labels = []

for label, group in df.groupby((df.index // SEQUENCE_LENGTH)):
    chunk = group.drop(columns=['label'])
    if len(chunk) == SEQUENCE_LENGTH:
        sequences.append(chunk.values)
        labels.append(group['label'].iloc[0])

X = np.array(sequences)
y = np.array(labels)

# --- ENCODING DELLE LABEL ---
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = to_categorical(y_encoded)

# --- SPLIT TRAIN/TEST ---
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# --- COSTRUISCI IL MODELLO LSTM ---
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, FEATURES_PER_FRAME)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(le.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# --- TRAINING ---
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2)

# --- VALUTAZIONE ---
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Accuracy finale: {acc:.2f}")

# --- SALVATAGGIO ---
model.save("pose_lstm_model.h5")
np.save("pose_labels.npy", le.classes_)
print("✅ Modello e label salvati.")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# --- PARAMETRI ---
SEQUENCE_LENGTH = 20
FEATURES_PER_FRAME = 132

# --- CARICA MODELLO E LABELS ---
model = load_model("pose_lstm_model.h5")
labels = np.load("pose_labels.npy")

# --- MEDIAPIPE SETUP ---
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# --- BUFFER PER I FRAME ---
sequence = []

# --- APERTURA WEBCAM ---
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Pre-elaborazione
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Estrai 33 landmark * (x,y,z,visibility)
        landmarks = results.pose_landmarks.landmark
        row = []
        for lm in landmarks:
            row.extend([lm.x, lm.y, lm.z, lm.visibility])
        sequence.append(row)

        # Disegna landmarks sul frame
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Se abbiamo una sequenza completa
        if len(sequence) == SEQUENCE_LENGTH:
            input_data = np.expand_dims(sequence, axis=0)  # shape: (1, 100, 132)
            prediction = model.predict(input_data, verbose=0)[0]
            predicted_label = labels[np.argmax(prediction)]

            # Mostra la label sul video
            cv2.putText(frame, f"Esercizio: {predicted_label}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

            # Rimuove il frame più vecchio (sliding window)
            sequence.pop(0)

    else:
        cv2.putText(frame, "Corpo non rilevato", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    # Mostra il frame
    cv2.imshow("Real-Time Exercise Classifier", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
